<a href="https://colab.research.google.com/github/Yeongchae-Shim/Bankruptcy-prediction/blob/master/openAI_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok xgboost tensorflow scikit-learn --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 681.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [2]:
# STEP 2: Streamlit 앱 코드 저장
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pyngrok import ngrok

st.set_page_config(page_title="VaR Prediction Dashboard", layout="centered")

st.title("📉 Portfolio VaR Predictor")
st.markdown("This dashboard predicts 1-day Value at Risk (VaR) using XGBoost and ANN.")

# Sample input
st.sidebar.header("Portfolio Input (Daily Returns)")
portfolio_input = st.sidebar.text_area("Enter comma-separated returns (e.g., -0.01, 0.02, -0.03)", "-0.01, 0.02, -0.03, 0.01, -0.015, 0.005")

try:
    returns = np.array([float(x) for x in portfolio_input.split(",")])
    returns = returns.reshape(-1, 1)
except:
    st.error("❌ Invalid input format. Please check your returns.")
    st.stop()

# Prepare features
df = pd.DataFrame(returns, columns=['Return'])
df['Lag1'] = df['Return'].shift(1)
df['Lag2'] = df['Return'].shift(2)
df['Lag3'] = df['Return'].shift(3)
df['MA_3'] = df['Return'].rolling(3).mean()
df['Vol_3'] = df['Return'].rolling(3).std()
df['Target'] = df['Return'].shift(-1)  # next-day return

df = df.dropna()

X = df[['Lag1', 'Lag2', 'Lag3', 'MA_3', 'Vol_3']]
y = df['Target']

# Split for simple test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- XGBoost ---
xgb_model = XGBRegressor(n_estimators=100, max_depth=3)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_var = np.percentile(xgb_pred, 5)

# --- ANN ---
ann_model = Sequential([
    Dense(32, input_shape=(X.shape[1],), activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])
ann_model.compile(optimizer='adam', loss='mse')
ann_model.fit(X_train, y_train, epochs=50, verbose=0)
ann_pred = ann_model.predict(X_test).flatten()
ann_var = np.percentile(ann_pred, 5)

st.metric("🔺 XGBoost VaR (95%)", f"{xgb_var:.2%}")
st.metric("🔷 ANN VaR (95%)", f"{ann_var:.2%}")

# Plot
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(xgb_pred, bins=30, alpha=0.5, label='XGBoost')
ax.hist(ann_pred, bins=30, alpha=0.5, label='ANN')
ax.axvline(xgb_var, color='red', linestyle='--', label=f'XGB VaR: {xgb_var:.2%}')
ax.axvline(ann_var, color='blue', linestyle='--', label=f'ANN VaR: {ann_var:.2%}')
ax.set_title("Predicted Return Distribution")
ax.legend()
st.pyplot(fig)

Writing app.py


## GPT

### Create Chat Completion

In [ ]:
chat = openai.ChatCompletion.create(
  model = "gpt-3.5-turbo",                             # 모델명 (필수)
  messages = [{"role": "user", "content": "난 뭘 하고 살면 좋을까?"}],
                                                       # 메세지 리스트 (필수 but "name") {"role": system/user/assistant, "content": 메시지 내용, "name": 이름}
  temperature = 1,                                     # output randomness [0~2] 0.2 small 0.8 big (either this or top_n, not both)
  # top_p = 1,                                         # probability considered [0~1]
  n = 1,                                               # N of chat completion choices
  stream = False,                                      # streaming
  # stop = ['.'],                                      # token 생성을 멈추게 하는 조건 sequences(4개까지)
  max_tokens = 200,                                    # 최대 token 수
  presence_penalty = 0,                                # 새로운 토픽을 말할 확률을 높여주는 [-2~2]
  frequency_penalty = 0,                               # 똑같은 문장을 말할 확률을 낮춰주는 [-2~2]
  logit_bias = {},                                     # 특정 토큰들의 출현 확률을 수정
  user = '',                                           # openAI 모니터용 ID
)
print(chat)

In [ ]:
print(chat["choices"][1]["message"]["content"])

## Whisper

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2023Spring/main/test.wav"
os.system("curl " + url + " > test.wav")

### Create Transcript

In [ ]:
audio_file = open("test.wav", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)
print(transcript)

In [ ]:
print(transcript['text'])

### Create Translation

In [ ]:
audio_file = open("test.wav", "rb")
translation = openai.Audio.translate("whisper-1", audio_file)
print(translation)

In [ ]:
print(translation['text'])

## DALL·E 2

### Create Image

In [ ]:
image = openai.Image.create(
  prompt="cat and dog",
  n=1,
  size="512x512"
)
print(image)

### Create Image Edit

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2023Spring/main/original.png"
os.system("curl " + url + " > original.png")
url = "https://raw.githubusercontent.com/hsnam95/class2023Spring/main/mask.png"
os.system("curl " + url + " > mask.png")

In [ ]:
image = openai.Image.create_edit(
  image=open("original.png", "rb"),
  mask=open("mask.png", "rb"),
  prompt="a greedy pig whose tongue is snake-like without the masked area",
  n=1,
  size="512x512"
)
print(image)

### Create Image Variation

In [ ]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2023Spring/main/cat_dot.png"
os.system("curl " + url + " > cat_dot.png")

In [ ]:
image = openai.Image.create_variation(
  image=open("cat_dot.png", "rb"),
  n=1,
  size="512x512"
)
print(image)